In [1]:
import numpy as np
import os
import sys
sys.path.append(r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project')

import matplotlib.pyplot as plt # for plotting
%matplotlib widget
import pandas as pd # for data manipulation
from Models.load_data import *
from Models.helpers import *

%load_ext autoreload
%autoreload 2

import torch

path = r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Training_data'
features_list = os.listdir(os.path.join(path, r'features'))

In [3]:
# This cell takes 19s
# Instantiate the dataset without transformations
dataset = IndependentCSVDataset(path, features_list)

# Split dataset into training and testing sets
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

feature_min, feature_max = compute_global_minmax(train_dataset)
print("Global feature min (train set only):", feature_min)
print("Global feature max (train set only):", feature_max)

global_transform = GlobalMinMaxNormalize(feature_min, feature_max)
train_dataset.dataset.transform = global_transform
test_dataset.dataset.transform = global_transform  # Same transform to prevent data leakage

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)



Skipping JETno97612.csv: sequence length 6001 is unexpected.
Skipping JETno87125.csv: sequence length 6001 is unexpected.
Skipping JETno91071.csv: sequence length 6001 is unexpected.
Skipping JETno99471.csv: sequence length 6001 is unexpected.
Processed 296 sequences with shape (60, 600).
Global feature min (train set only): [ 0.0000000e+00 -3.2111890e+06 -9.1752766e+13  0.0000000e+00
 -8.4846081e+13  0.0000000e+00  0.0000000e+00 -3.2102978e+06
 -9.2097940e+13  0.0000000e+00 -7.3527542e+13  0.0000000e+00
  0.0000000e+00 -3.2099495e+06 -9.2103753e+13  0.0000000e+00
 -7.2064157e+13  0.0000000e+00  0.0000000e+00 -3.2106140e+06
 -9.1307901e+13  0.0000000e+00 -9.4136540e+13  0.0000000e+00
  0.0000000e+00 -3.2105688e+06 -9.0980284e+13  0.0000000e+00
 -8.5559138e+13  0.0000000e+00  0.0000000e+00 -3.2106132e+06
 -9.0181126e+13  0.0000000e+00 -9.5513295e+13  0.0000000e+00
  0.0000000e+00 -3.2092092e+06 -8.9826750e+13  0.0000000e+00
 -7.5441352e+13  0.0000000e+00  0.0000000e+00 -3.2083680e+06
 -

In [4]:
# Create an iterator from the DataLoader
train_iterator = iter(train_loader)

# Get a single batch
batch = next(train_iterator)

# Unpack the batch (assuming your dataset returns (inputs, targets))
inputs, targets = batch

print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)  # (60, 600)


Inputs shape: torch.Size([32, 60, 600])
Targets shape: torch.Size([32, 60, 100])


In [5]:
# Define loss and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss(pos_weight=compute_class_weights(train_loader))
# Define the model and print the architecture, example with dummy input shapes

input_size = 600       # Number of features per time step
hidden_size = 400     # Number of LSTM hidden units
num_layers = 2       # Number of LSTM layers
output_size = 100      # Binary classification
dropout = 0.2

# Instantiate the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

# Print model architecture
print(model)

# Example dummy input: batch_size=32, sequence_length=6000, input_size=6
dummy_input = inputs
dummy_output = model(dummy_input)

print("Input shape:", dummy_input.shape)  # Expected: (32, 6000, 6) -> 6 features per timestep
print("Output shape:", dummy_output.shape)  # Expected: (32, 6000,) -> one binary value per timestep
print("Targets shape:", targets.shape)

loss = criterion(dummy_output, targets)

print(loss.item())

Class counts: [1397912.0, 18088.0]
Class ratio: 77.28394317626953
LSTMModel(
  (lstm): LSTM(600, 400, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=400, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=100, bias=True)
  (sigmoid): Sigmoid()
)


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:230: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(class_counts[0] / class_counts[1], dtype=torch.float32)


Input shape: torch.Size([32, 60, 600])
Output shape: torch.Size([32, 60, 100])
Targets shape: torch.Size([32, 60, 100])
1.0589429140090942


In [ ]:
# Define the optimizer
import torch
import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=5e-2)

# Assume `train_loader` is a DataLoader yielding batches of (input, target)
num_epochs = 20

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        print(f'Accumulated epoch loss: {epoch_loss:.4f}, {batch_idx=}', end='\r')
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)  # (60, 600)


Epoch [1/20], Loss: 5.1517.2139, batch_idx=7
Epoch [2/20], Loss: 1.3213.5708, batch_idx=7


In [32]:
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score

def evaluate_test_metrics(model, test_loader, criterion, device, threshold=0.5):
    """
    Evaluates the test loss, F1-score, and ROC-AUC of a trained model.

    Parameters:
    - model: The trained PyTorch model.
    - test_loader: DataLoader for the test dataset.
    - criterion: Loss function (e.g., nn.BCEWithLogitsLoss).
    - device: The device ('cuda' or 'cpu').
    - threshold: Decision threshold for binary classification (default=0.5).

    Returns:
    - Average test loss
    - F1-score
    - ROC-AUC score
    """
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    total_samples = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to device

            # Forward pass
            outputs = model(inputs)  # (batch_size, 6000) -> raw logits

            # Compute loss
            loss = criterion(outputs, targets)  # BCEWithLogitsLoss expects raw logits
            test_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)

            # Convert logits to probabilities using Sigmoid
            probs = torch.sigmoid(outputs)  # (batch_size, 6000) -> probabilities in [0,1]

            # Apply threshold to get binary predictions
            preds = (probs > threshold).int()  # Convert to 0 or 1

            # Flatten for metric calculations
            all_targets.extend(targets.cpu().numpy().flatten())  # Convert to 1D list
            all_predictions.extend(preds.cpu().numpy().flatten())  # Convert to 1D list

    # Compute average loss
    avg_test_loss = test_loss / total_samples

    # Compute F1-score
    f1 = f1_score(all_targets, all_predictions, average='macro')

    return avg_test_loss, f1

# Example Usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loss, test_f1 = evaluate_test_metrics(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"F1 Score: {test_f1:.4f}")


Test Loss: 8.1832
F1 Score: 0.0170
